In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask.dataframe as dd
from time import time
import gc #Coletor de lixo
gc.enable()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Os tipos de dados foram tirados daqui: https://www.kaggle.com/code/rohanrao/tutorial-on-reading-large-datasets/notebook
dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
}

## TAREFA 1: LENDO O DATAFRAME EM CSV

* Pandas

In [ ]:
inicio = time()

pdf = pd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", nrows = 10000, dtype=dtypes)
final = time()

print(f"Lendo o dataframe com pandas (10000 linhas): {(final-inicio):.3f}s")

In [ ]:
inicio = time()

pdf = pd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", nrows = 100000, dtype=dtypes)
final = time()

print(f"Lendo o dataframe com pandas (100000 linhas): {(final-inicio):.3f}s")

In [ ]:
inicio = time()

pdf = pd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", nrows = 1000000, dtype=dtypes)
final = time()

print(f"Lendo o dataframe com pandas (1000000 linhas): {(final-inicio):.3f}s")

* Dask

In [ ]:
inicio = time()

pdf = dd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", blocksize = "30MB", dtype=dtypes).head(n = 10000)
final = time()

print(f"Lendo o dataframe com dask (10000 linhas): {(final-inicio):.3f}s")

In [ ]:
inicio = time()

pdf = dd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", blocksize = "64MB", dtype=dtypes).head(n = 100000, npartitions = 20)
final = time()

print(f"Lendo o dataframe com dask (100000 linhas): {(final-inicio):.3f}s")

In [ ]:
#Ao tentar rodar as linhas abaixo, obtenho o erro: "Your notebook tried to allocate more memory than is available. It has restarted."
#inicio = time()

#pdf = dd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", blocksize = None).head(n = 1000000)
#final = time()

#print(f"Lendo o dataframe com dask (1000000 linhas): {(final-inicio):.3f}s")

In [ ]:
inicio = time()

pdf = dd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", dtype=dtypes).head(n = 1000000, npartitions = 48)
final = time()

print(f"Lendo o dataframe com dask (1000000 linhas): {(final-inicio):.3f}s")

## TAREFA 2a: LENDO O DATAFRAME EM PARQUET 1 ##

O dataset foi criado aqui: https://www.kaggle.com/code/odins0n/load-parquet-files-with-low-memory/

As colunas originais float64 foram reduzidas para float32

O read_parquet do pandas não suporta o argumento nrows, nem dtype

In [ ]:
#Ao tentar rodar as linhas abaixo, obtenho o erro: "Your notebook tried to allocate more memory than is available. It has restarted."
#inicio = time()

#pdf = pd.read_parquet('../input/amex-parquet/train_data.parquet')
#final = time()

#print(f"Lendo o dataframe com pandas: {(final-inicio):.3f}s")

In [ ]:
inicio = time()

pdf = dd.read_parquet('../input/amex-parquet/train_data.parquet')
final = time()

print(f"Lendo o dataframe com dask: {(final-inicio):.3f}s")

## TAREFA 2b: LENDO O DATAFRAME EM PARQUET 2 ##

O dataset foi criado aqui: https://www.kaggle.com/competitions/amex-default-prediction/discussion/327908

As colunas originais float64 foram reduzidas para float32
Customer_id foi para string
S_2 foi para datetime
Todas as categóricas foram para category

O read_parquet do pandas não suporta o argumento nrows, nem dtype

In [ ]:
#Ao tentar rodar as linhas abaixo, obtenho o erro: "Your notebook tried to allocate more memory than is available. It has restarted."
inicio = time()

pdf = pd.read_parquet('/kaggle/input/american-express-default-prediction-snappy-parquet/train_data.snappy.parquet')
final = time()

print(f"Lendo o dataframe com pandas: {(final-inicio):.3f}s")

In [ ]:
inicio = time()

pdf = dd.read_parquet('/kaggle/input/american-express-default-prediction-snappy-parquet/train_data.snappy.parquet')
final = time()

print(f"Lendo o dataframe com dask: {(final-inicio):.3f}s")

## TAREFA 3: LENDO O DATAFRAME EM PICKLE

Baseado nesse notebook: https://www.kaggle.com/code/kingychiu/amex-woe-baseline-with-id-encoded-fp16-dataset

Ele transformou as float64 para float32 e usou LabelEncoder em customer_id

In [ ]:
inicio = time()

pdf = pd.read_pickle('../input/ae-credit-id-encoded-dataset-fp32/id_encoded_fp32_train_data.pkl')
final = time()

print(f"Lendo o dataframe com pandas: {(final-inicio):.3f}s")

In [ ]:
#DASK NÃO SUPORTA O FORMATO PICKLE
#inicio = time()

#pdf = dd.read_pickle('../input/ae-credit-id-encoded-dataset-fp32/id_encoded_fp32_train_data.pkl')
#final = time()

#print(f"Lendo o dataframe com dask: {(final-inicio):.3f}s")

* Vencedor: dask + formato parquet.